In [ ]:
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline
def sen(Y_test,Y_pred,n):
    
    sen = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        sen1 = tp / (tp + fn)
        sen.append(sen1)
        
    return sen
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe
def readfile(file):  

    f = h5py.File(file,'r')
    f.keys() 

    eeg1=f['eeg1'][:]
    eeg2=f['eeg2'][:]
    emg = f['emg'][:]
    scores=f['heshui'][:]
    times=f['times'][:]
    emg=np.array(emg).astype(np.float16)
    eeg1=np.array(eeg1).astype(np.float16)
    eeg2=np.array(eeg2).astype(np.float16)
    times=np.array(times).astype(np.float16)
    f.close()
    gc.collect()
    return emg,eeg1,eeg2,scores,times
def readfilez(file):  

    f = h5py.File(file,'r')
    f.keys() 
    eegz=f['z'][:]   
    eeg1=np.array(eegz[:,:30]).astype(np.float16)
    eeg2=np.array(eegz[:,30:60]).astype(np.float16)
    emg=np.array(eegz[:,60:90]).astype(np.float16)
    scores=np.array(eegz[:,90:91]).astype(np.float16)
    f.close()
    gc.collect()
    return emg,eeg1,eeg2,scores
plt.rcParams['font.sans-serif'] = ['SimHei']


def sen(Y_test,Y_pred,n):#n为分类数
    
    sen = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        sen1 = tp / (tp + fn)
        sen.append(sen1)
        
    return sen
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe

def fill_plot(interp_tpr,name,aucs):
    colors = ['#7E4D99', '#53A362','#DA3F34']   
    target_names = ['Wakefulness', 'NREM', 'REM']
    interp_tpr=np.array(interp_tpr)    
    fig, ax = plt.subplots(figsize=(8, 8))
    mean_tpr = np.mean(interp_tpr[:,0,:], axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(interp_tpr[:,0,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[0],
        label=f"Mean ROC of '{target_names[0]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[0], alpha=0.2,label=r"95% Confidence interval")  
    
    mean_tpr = np.mean(interp_tpr[:,1,:], axis=0)
    std_tpr = np.std(interp_tpr[:,1,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[1],
        label=f"Mean ROC of '{target_names[1]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[1], alpha=0.2,label=r"95% Confidence interval")  
    
    
    mean_tpr = np.mean(interp_tpr[:,2,:], axis=0)
    std_tpr = np.std(interp_tpr[:,2,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[2],
        label=f"Mean ROC for '{target_names[2]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[2], alpha=0.1,label=r"95% Confidence interval")   
    

    ax.set_xlabel("False Positive Rate (1 - Specificity)", fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel("True Positive Rate (Sensitivity)", fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    plt.xticks(fontname='Times New Roman', fontsize=16,weight='bold')
    plt.yticks(fontname='Times New Roman', fontsize=16,weight='bold')
    ax.legend(loc="lower right")
    ax.legend(prop={'family': 'Times New Roman', 'size': 14,'weight':'bold'}) 
    plt.savefig('E:/fenqi/{}'.format(name),dpi=300,bbox_inches = 'tight')
    plt.close()
#     plt.show()
    

def tcplot(eeg2f,eeg2fyc,tab,tabyc,can,name,num):
    can=dict(can)

    ez=eeg2f
    ezyc=eeg2fyc
    le = LabelEncoder()
    tab = le.fit_transform(tab)
    tabyc = le.fit_transform(tabyc)
    x_train,y_train=np.array(ez),np.array(tab)
    x_test,y_test=np.array(ezyc),np.array(tabyc)
    model = LGBMClassifier(learning_rate=can['learning_rate'],num_leaves=can['num_leaves'],
                           max_depth=can['max_depth'],min_child_samples=can['min_child_samples'],
                           max_bin=can['max_bin'],subsample=can['subsample'],
                           colsample_bytree=can['colsample_bytree'],min_child_weight=can['min_child_weight'],
                           reg_lambda=can['reg_lambda'],reg_alpha=can['reg_alpha'],
                           scale_pos_weight=can['scale_pos_weight'],n_estimators=can['n_estimators'],verbose=-1)

    model.fit(x_train, y_train)

    pred=model.predict(x_test)
    preds=model.predict_proba(x_test)
    score = model.score(x_test, y_test)

    target_names = ['Wakefulness', 'NREM', 'REM']

    y_test1 = label_binarize(y_test, classes=[0, 1, 2])

    fpr={}
    tpr={}
    roc_auc={}
    rocz=[]
    f_head2=[]
    interp_tprf=[]

    pres=[]
    recal=[]
    sens=[]
    spec=[]

    pres.append(precision_score(y_test, pred,average=None))
    recal.append(recall_score(y_test, pred,average=None))
    sens.append(sen(y_test, pred,3))
    spec.append(spe(y_test, pred,3))

    for i in range(3):
    #         print(i)
        fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], preds[:, i])
        rocz.append(auc(fpr[i], tpr[i]))
        roc_auc[i] = auc(fpr[i], tpr[i])
#         print("roc_auc:",roc_auc)
        interp_tprff=np.interp(mean_fpr, fpr[i], tpr[i])
        interp_tprf.append(interp_tprff)
    df_rocz = pd.DataFrame(rocz, columns=['AUC'])
    df_sleep_stages = pd.DataFrame(target_names, columns=['Sleep Stage'])
    dfauc = pd.concat([df_sleep_stages, df_rocz], axis=1)    

    pres=np.array(pres)
    recal=np.array(recal)
    sens=np.array(sens)
    spec=np.array(spec)
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=pres[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=pres[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=pres[:,2]
    df2['Sleep_Stage']=['Rem']
    dfj=pd.DataFrame()
    dfj=dfj.append(df)
    dfj=dfj.append(df1)
    dfj=dfj.append(df2)
    dfj['Cata']='Precision'
    dfj['Sample_number']=num
    dfj['Area']=name
    
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=recal[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=recal[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=recal[:,2]
    df2['Sleep_Stage']=['Rem']
    dfr=pd.DataFrame()
    dfr=dfr.append(df)
    dfr=dfr.append(df1)
    dfr=dfr.append(df2)
    dfr['Cata']='Recall'
    dfr['Sample_number']=num
    dfr['Area']=name    
    
    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=sens[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=sens[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=sens[:,2]
    df2['Sleep_Stage']=['Rem']
    dfm=pd.DataFrame()
    dfm=dfm.append(df)
    dfm=dfm.append(df1)
    dfm=dfm.append(df2)
    dfm['Cata']='Sensitivity'
    dfm['Sample_number']=num
    dfm['Area']=name    

    df=pd.DataFrame()
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    df['Data']=spec[:,0]
    df['Sleep_Stage']=['Wakefulness']
    df1['Data']=spec[:,1]
    df1['Sleep_Stage']=['Nrem']
    df2['Data']=spec[:,2]
    df2['Sleep_Stage']=['Rem']
    dft=pd.DataFrame()
    dft=dft.append(df)
    dft=dft.append(df1)
    dft=dft.append(df2)
    dft['Cata']='Specificity'
    dft['Sample_number']=num
    dft['Area']=name
    
    dfz=pd.DataFrame()
    dfz=dfz.append(dfj)
    dfz=dfz.append(dfr)
    dfz=dfz.append(dfm)
    dfz=dfz.append(dft)
    return interp_tprf,score,dfz,dfauc

def xl_zuhe(num,lgbcan,pbar):
    interp_tprd=[]
    interp_tprx=[]
    interp_tprj=[]
    interp_tprdx=[]
    interp_tprdxj=[]
    interp_tprdj=[]    

    scored,scorex,scorej,scoredx,scoredxj,scoredj=[],[],[],[],[],[]
    dfzz=pd.DataFrame()
    dfdi=pd.DataFrame()
    dfxi=pd.DataFrame()
    dfji=pd.DataFrame()
    dfdx=pd.DataFrame()
    dfdj=pd.DataFrame()
    dfdxj=pd.DataFrame()
    dfazd=pd.DataFrame()
    dfazx=pd.DataFrame()
    dfazj=pd.DataFrame()
    dfa=pd.DataFrame()
    for j in range(20):
        info=os.listdir(f_h5z)

        infoxl = np.random.choice(info,num, replace=False)
        infoyc =np.setdiff1d(info, infoxl)
        tab=[]
        eeg1f,eeg2f,emgf=[],[],[]
        for i in range(np.shape(infoxl)[0]):

            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoxl[i])

            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1f.extend(eeg1)
            eeg2f.extend(eeg2)
            emgf.extend(emg)
            tab.extend(scores)

        tabyc=[]
        eeg1fyc,eeg2fyc,emgfyc=[],[],[]
        
        for i in range(np.shape(infoyc)[0]):

            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoyc[i])
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1fyc.extend(eeg1)
            eeg2fyc.extend(eeg2)
            emgfyc.extend(emg)
            tabyc.extend(scores)

        interp_tprl,score,dfz,dfauc=tcplot(eeg2f,eeg2fyc,tab,tabyc,lgbcan,"Cerebral EEG",num)
        interp_tprd.append(interp_tprl)
        scored.append(score)
        dfazd= dfazd.append(dfauc)       

        dfzz= dfzz.append(dfz)

        interp_tprl,score,dfz,dfauc=tcplot(eeg1f,eeg1fyc,tab,tabyc,lgbcan,"Cerebellar EEG",num)
        interp_tprx.append(interp_tprl)
        scorex.append(score)
        dfzz= dfzz.append(dfz)
        dfazx= dfazx.append(dfauc)

        interp_tprl,score,dfz,dfauc=tcplot(emgf,emgfyc,tab,tabyc,lgbcan,"EMG",num)
        interp_tprj.append(interp_tprl)
        scorej.append(score)
        dfzz= dfzz.append(dfz)
        dfazj= dfazj.append(dfauc)
        
        pbar.update() 
#########################################################
    dfazd['Area']="Cerebral EEG"
    dfazx['Area']="Cerebellar EEG"
    dfazj['Area']="EMG"
    dfa=dfa.append(dfazd) 
    dfa=dfa.append(dfazx)
    dfa=dfa.append(dfazj)
    dfa.to_csv(f_sv+"{}AUC.csv".format(num))    
#########################################################
    np.save(f_sv+'{}Cerebral_tpr.npy'.format(num),interp_tprd)
    np.save(f_sv+'{}Cerebral_score.npy'.format(num),scored)
    fill_plot(interp_tprd,"{}Cerebral".format(num),scored)
#########################################################

    np.save(f_sv+'{}Cerebellar_tpr.npy'.format(num),interp_tprx)
    np.save(f_sv+'{}Cerebellar_score.npy'.format(num),scorex)
    fill_plot(interp_tprx,"{}Cerebellum".format(num),scorex)
#########################################################

    np.save(f_sv+'{}EMG_tpr.npy'.format(num),interp_tprj)
    np.save(f_sv+'{}EMG_score.npy'.format(num),scorej)
    fill_plot(interp_tprj,"{}EMG".format(num),scorej)
#########################################################
    return dfzz



In [ ]:

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from skopt.callbacks import VerboseCallback, DeadlineStopper, DeltaXStopper

def tiaocanlgb(freq, sh):
    x_true = np.array(freq)
    y_true = np.array(sh).astype(int)
    x_trainy, y_trainy = x_true, y_true
    le = LabelEncoder()
    y_trainy = le.fit_transform(y_trainy)
    print(y_trainy)
    print(x_trainy.shape)
    bayes_params = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (2, 50),      
        'max_depth': (0, 50),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 20, 'log-uniform'),
        'n_estimators': (50, 1500),
    }

    model = LGBMClassifier()
    kflod = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

    bayes_opt = BayesSearchCV(estimator=model, search_spaces=bayes_params, cv=kflod, n_jobs=-1, n_iter=32)
    bayes_opt.fit(x_trainy, y_trainy,callback=[VerboseCallback(100), DeadlineStopper(60*5), DeltaXStopper(0.001)])
    best_params = bayes_opt.best_params_
    print('贝叶斯优化的最佳参数：{0}'.format(best_params))

    return bayes_opt.best_params_

f_h5z="E:/fenqi/h5zz"
f_sv="E:/fenqi/result/"
mean_fpr = np.linspace(0, 1, 100)
info=os.listdir(f_h5z)
print(info)
tab=[]
eeg1f,eeg2f,emgf=[],[],[]
for i in range(np.shape(info)[0]):
    print(info[i])
    print("xxxxxxxxxx")
    domain = os.path.abspath(f_h5z)
    infor = os.path.join(domain,info[i])
    print(infor)
    emg,eeg1,eeg2,scores=readfilez(infor)
    eeg1f.extend(eeg1)
    eeg2f.extend(eeg2)
    emgf.extend(emg)
    tab.extend(scores)
bcanshu=tiaocanlgb(eeg2f, tab)

In [ ]:

from tqdm import tqdm
import ctypes
ES_DISPLAY_REQUIRED  = 0x00000002
ES_CONTINUOUS        = 0x80000000
def set_display_required():
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_DISPLAY_REQUIRED)
set_display_required()

nums=[5,10,15]
total = len(nums) * 20
# 创建一个进度条
pbar = tqdm(total=total)
dfzzz=pd.DataFrame()
mean_fpr = np.linspace(0, 1, 100)
for i in (nums):
    dfzz=xl_zuhe(i,bcanshu,pbar)
    dfzzz=dfzzz.append(dfzz)
pbar.close()
dfzzz.to_csv(f_sv+"cerebral_eeg&cerebellar_eeg&emg_report.csv")

In [ ]:
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline

dfzzz=pd.read_csv(fcsv)
dfzzz = dfzzz[dfzzz['Area'].isin(["Cerebral EEG", "Cerebellar EEG", "EMG"])]

In [ ]:
sns.set_theme(style="whitegrid",font='Times New Roman',font_scale=2,rc={'font.weight': 'bold'})

colors=['#C49A98', '#7E4D99', '#3B84C4','#E6873E','#53A362','#DA3F34']
#presicion
g = sns.catplot(
    data=dfzzz[dfzzz['Cata']=="Precision"], x='Sample_number', y='Data', hue="Area", col="Sleep_Stage",
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Precision rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.0,1.2,0.2))
    ax.set_ylim(-0.2,1.19)
g._legend.set_title("")
g.set_titles(col_template='')
g.despine(left=True)

plt.savefig(fsv+'Precision.jpg',dpi=300,bbox_inches = 'tight')


#Recall
g = sns.catplot(
    data=dfzzz[dfzzz['Cata']=="Recall"], x='Sample_number', y='Data', hue="Area", col="Sleep_Stage",
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Recall/sensitivity rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.0,1.2,0.2))
    ax.set_ylim(-0.2,1.19)

g._legend.set_title("")
g.set_titles(col_template='')
g.despine(left=True)
plt.savefig(fsv+'Recall.jpg',dpi=300,bbox_inches = 'tight')

#Specificity
g = sns.catplot(
    data=dfzzz[dfzzz['Cata']=="Specificity"], x='Sample_number', y='Data', hue="Area", col="Sleep_Stage",
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)
for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel('Specificity rate', fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_yticks(np.arange(0.0,1.2,0.2))
    ax.set_ylim(-0.2,1.19)
g._legend.set_title("")
g.set_titles(col_template='')
g.despine(left=True)
plt.savefig(fsv+'Specificity.jpg',dpi=300,bbox_inches = 'tight')

In [ ]:
from scipy import stats 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
pd.set_option('display.float_format',lambda x : '%.3f' % x)
np.set_printoptions(suppress=True)
def print_pvalues_and_stars(df):
    res = pairwise_tukeyhsd(df['value'], df['Area'])
    comparisons = res._results_table.data[1:]

    for comparison in comparisons:
        print(comparison)
        group1, group2, pvalue = comparison[0], comparison[1], comparison[3]
        stars = ''
        if pvalue < 0.001: stars = '***'
        elif pvalue < 0.01: stars = '**'
        elif pvalue < 0.05: stars = '*'
        print(f"{group1} vs {group2}: pvalue = {pvalue:.4f} {stars}")

def anova_tuk(name,num):

    dftest=dfzzz[(dfzzz['Cata']==name) & (dfzzz['Sample_number']==num)& (dfzzz['Sleep_Stage']=='Wakefulness')].melt('Area','Data')
    modeltest=ols('value~C(Area)',dftest).fit()
    anova_table=anova_lm(modeltest)
    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    
    print_pvalues_and_stars(dftest)

    dftest=dfzzz[(dfzzz['Cata']==name) & (dfzzz['Sample_number']==num)& (dfzzz['Sleep_Stage']=='Nrem')].melt('Area','Data')
    modeltest=ols('value~C(Area)',dftest).fit()
    anova_table=anova_lm(modeltest)
    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    
    print_pvalues_and_stars(dftest)
    dftest=dfzzz[(dfzzz['Cata']==name) & (dfzzz['Sample_number']==num)& (dfzzz['Sleep_Stage']=='Rem')].melt('Area','Data')
    modeltest=ols('value~C(Area)',dftest).fit()
    anova_table=anova_lm(modeltest)
    print(anova_table['F'][0])    
    print(np.float16(anova_table['PR(>F)'][0]))    

    print_pvalues_and_stars(dftest)

In [ ]:
flist=[5,10,15]
for i in (flist):
    print('xxxxxxxxxx')
    anova_tuk("Precision",i)
for i in (flist):
    print('xxxxxxxxxx')
    anova_tuk("Recall",i)
for i in (flist):
    print('xxxxxxxxxx')    
    anova_tuk("Sensitivity",i)
for i in (flist):
    print('xxxxxxxxxx')    
    anova_tuk("Specificity",i)


In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def fill_plot(interp_tpr,name,aucs):
    colors = ['#7E4D99', '#53A362','#DA3F34']   
    target_names = ['Wakefulness', 'NREM', 'REM']
    interp_tpr=np.array(interp_tpr)
    fig, ax = plt.subplots(figsize=(8, 8))
    mean_tpr = np.mean(interp_tpr[:,0,:], axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(interp_tpr[:,0,:], axis=0)

    mean_auc = np.mean(interp_tpr[:,0,:])
    std_auc = np.std(interp_tpr[:,0,:])   
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[0],
        label=f"Mean ROC of '{target_names[0]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[0], alpha=0.2,label=r"95% Confidence interval")  

    mean_tpr = np.mean(interp_tpr[:,1,:], axis=0)
    std_tpr = np.std(interp_tpr[:,1,:], axis=0)

    mean_auc = np.mean(interp_tpr[:,1,:])
    std_auc = np.std(interp_tpr[:,1,:])
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[1],
        label=f"Mean ROC of '{target_names[1]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[1], alpha=0.2,label=r"95% Confidence interval")  
     
    mean_tpr = np.mean(interp_tpr[:,2,:], axis=0)
    std_tpr = np.std(interp_tpr[:,2,:], axis=0)

    mean_auc = np.mean(interp_tpr[:,2,:])
    std_auc = np.std(interp_tpr[:,2,:])   
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[2],
        label=f"Mean ROC for '{target_names[2]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[2], alpha=0.1,label=r"95% Confidence interval")   
    

    ax.set_xlabel("False Positive Rate (1 - Specificity)", fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    ax.set_ylabel("True Positive Rate (Sensitivity)", fontdict={'family': 'Times New Roman', 'size': 26,'weight':'bold'})
    plt.xticks(fontname='Times New Roman', fontsize=16,weight='bold')
    plt.yticks(fontname='Times New Roman', fontsize=16,weight='bold')
    ax.legend(loc="lower right")
    ax.legend(prop={'family': 'Times New Roman', 'size': 14,'weight':'bold'}) 
    plt.savefig(fsv+'{}'.format(name),dpi=300,bbox_inches = 'tight')
    plt.close()
    print(np.shape(interp_tpr[:,0,:]))
xssv=f_sv+"5Cerebellar_score.npy"
xtsv=f_sv+"5Cerebellar_tpr.npy"
dssv=f_sv+"5Cerebral_score.npy"
dtsv=f_sv+"5Cerebral_tpr.npy"
jssv=f_sv+"5EMG_score.npy"
jtsv=f_sv+"5EMG_tpr.npy"
fsv=f_sv
mean_fpr = np.linspace(0, 1, 100)
for i in [5,10,15]:
    xssv=f_sv+"{}Cerebellar_score.npy".format(i)
    xtsv=f_sv+"{}Cerebellar_tpr.npy".format(i)
    dssv=f_sv+"{}Cerebral_score.npy".format(i)
    dtsv=f_sv+"{}Cerebral_tpr.npy".format(i)
    jssv=f_sv+"{}EMG_score.npy".format(i)
    jtsv=f_sv+"{}EMG_tpr.npy".format(i)
    scorex= np.load(xssv)
    tprx= np.load(xtsv)
    scored= np.load(dssv)
    tprd= np.load(dtsv)
    scorej= np.load(jssv)
    tprj= np.load(jtsv)
    fill_plot(tprx,'{}Cerebellar EEG'.format(i),scorex)
    fill_plot(tprd,'{}Cerebral EEG'.format(i),scored)
    fill_plot(tprj,'{}EMG'.format(i),scorej)

In [ ]:
dfz=pd.DataFrame()
for i in [5,10,15]:
    fcsv=f_sv+"{}AUC.csv".format(i)
    target_names = ['Wakefulness', 'NREM', 'REM']
    Area=['Cerebellar EEG','Cerebral EEG','EMG']
    df=pd.read_csv(fcsv)
    df['Sample_number']=i
    dfz=dfz.append(df)
dfz= dfz.drop(df.columns[0], axis=1)    
sns.set_theme(style="whitegrid",font='Times New Roman',font_scale=1.5,rc={'font.weight': 'bold'})
colors=['#C49A98', '#7E4D99', '#3B84C4','#E6873E','#53A362','#DA3F34']
g = sns.catplot(
    data=dfz, x='Sample_number', y='AUC', hue="Area", col="Sleep Stage",
    palette=colors, errorbar="se",scale='width',
    kind="violin"
)

for ax in g.axes.flat:
    ax.set_xticklabels([])
    ax.set_xlabel('', fontdict={'family': 'Times New Roman', 'size': 18,'weight':'bold'})
    ax.set_ylabel('Mean AUC (%)', fontdict={'family': 'Times New Roman', 'size': 18,'weight':'bold'})
    ax.set_yticks(np.arange(0.0,1.2,0.2))
    ax.set_ylim(-0.2,1.19)
    ax.tick_params(labelsize=14)

g._legend.set_title("")
g.set_titles(col_template='')
g.despine(left=True)
plt.savefig(fsv+'AUC.jpg',dpi=300,bbox_inches = 'tight')